 ## EDA 

In [ ]:
# Data analysis and wrangling
import numpy as np
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocessing and model selection
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Matplotlib configuration
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Warnings configuration
import warnings
warnings.filterwarnings('ignore')


1. __Cargar datos__


Cargamos el conjunto de datos desde el archivo CSV


In [ ]:
df = pd.read_csv("../data/attrition_availabledata_03.csv")

2. __Exploración inicial__

En esta sección revisamos la estructura general del dataset


In [ ]:
print(df.info())

In [ ]:
dataset_shape = df.shape
print(f"El dataset contiene {dataset_shape[0]} filas y {dataset_shape[1]} columnas.")

In [ ]:
df.describe()

In [ ]:
df.head()

3. __Identificamos las variables categóricas y numéricas__

In [ ]:
# Identificar variables categóricas y numéricas
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
print("Variables categóricas:", categorical_columns)
print("Variables numéricas:", numerical_columns)

- Reclasificamos las variables añadiendo ordinales

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

# Eliminamos de numéricas las que hemos clasificado como ordinales
numerical_columns = [col for col in numerical_columns if col not in ordinal_columns]

print("Variables categóricas:", categorical_columns)
print("Variables ordinales:", ordinal_columns)
print("Variables numéricas:", numerical_columns)


4. __Análisis de la variable objetivo__

Revisamos la distribución de la variable objetivo para identificar desbalance de clases


In [ ]:
# Balance de la variable objetivo "Attrition"
if "Attrition" in df.columns:
    plt.figure(figsize=(4,4))
    sns.countplot(x=df["Attrition"], palette="viridis")
    plt.title("Distribución de la variable objetivo (Attrition)\n")
    plt.show()
    
    attrition_counts = df["Attrition"].value_counts(normalize=True)
    display(pd.DataFrame(attrition_counts).rename(columns={"Attrition": "Proportion"}).reset_index(drop=True)*100)


In [ ]:
df.Attrition.value_counts().sort_index().to_frame()

 El dataset está desbalanceado, con 2466 empleados que no abandonan la empresa (NO) y 474 que sí lo hacen (SI). Esto podría afectar el rendimiento de los modelos predictivos, favoreciendo la clase mayoritaria. 



5. __Identificar valores nulos__

In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].to_frame().reset_index()
missing_values.columns = ["Column Name", "Missing Values"]

display(missing_values)

6. __Identificar columnas con valores únicos (constantes)__

In [ ]:
unique_values = df.nunique()

constant_columns = df.nunique()[df.nunique() == 1].to_frame().reset_index()
constant_columns.columns = ["Column Name", "Unique Value Count"]
constant_columns["Unique Value"] = constant_columns["Column Name"].apply(lambda col: df[col].unique()[0])

display(constant_columns)


- Eliminar las columnas constantes

In [ ]:
df_numeric = df.select_dtypes(include=['number'])

constant_columns_list = constant_columns["Column Name"].tolist()
df_numeric = df_numeric.drop(columns=constant_columns_list, errors='ignore')


7. __Crear matriz de correlación__

In [ ]:
correlation_matrix = df_numeric.corr()

plt.figure(figsize=(18, 18))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación (Sin Columnas Constantes)")
plt.show()



8. __Histogramas de Variables Numéricas__

Visualizamos la distribución de las variables numéricas mediante histogramas

In [ ]:
plt.figure(figsize=(15, 12))
df[numerical_columns].hist(bins=30, figsize=(15, 20), layout=(len(numerical_columns)//3 + 1, 3))
plt.suptitle("Histogramas de Variables Numéricas", fontsize=16)
plt.show()